In [ ]:
!uv pip install -U Ipython
!uv pip install webvtt-py sacrebleu rouge_score pycocoevalcap
!uv pip install peft hf_xet hf-trim evaluate
!pip install git+https://github.com/google-research/bleurt.git

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/e2e-slt-streaming
%load_ext autoreload
%autoreload 2

# !wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip -O ./checkpoints/BLEURT-20.zip
# !git fetch origin && git reset --hard origin/main # Don't care about local changes and overwrite everything
# !git pull
!ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1AlNQaxmBldq_vhxtUYkHpWaALnGeTKc4/e2e-slt-streaming
backbones    deformable_detr  __init__.py  poses	     train_two_stage.py
captioners   docs	      loader.py    postprocess.py    utils.py
checkpoints  eval.py	      loss.py	   __pycache__	     wandb
config.py    evaluation       main.py	   README.md
dataset      experiments      pdvc.py	   requirements.txt


In [ ]:
# Take around 14-16mins on A100, 30mins on T4
!rm -rf /tmp/auto_sat_aligned
!rm -rf /tmp/manual_sat_aligned
!rm -rf /tmp/bobsl_dwpose
!7z x ./dataset/auto_sat_aligned.zip -o/tmp
!7z x ./dataset/manual_sat_aligned.zip -o/tmp
!7z x ./dataset/BOBSL/bobsl_dwpose.zip -o/tmp
!7z x ./checkpoints/BLEURT-20.zip -o/tmp
# !python3 captioners/trim_mbart.py # Prepare trimmed MBart tokenizer/model

In [ ]:
%%time
!python3 main.py \
	--mode 1 \
	--output_dir /tmp/mode1 \
	--max_event_tokens 40 \
	--d_model 1024 \
	--encoder_layers 2 \
	--decoder_layers 2 \
	--num_cap_layers 3 \
	--num_queries 30 \
	--num_train_epochs 64 \
	--learning_rate 5e-4 \
    --weight_decay 0.05 \
	--per_device_train_batch_size 32

2026-01-04 19:11:57.400547: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-04 19:11:57.422013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767553917.445028  340033 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767553917.452175  340033 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767553917.471717  340033 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
%%time
!python3 main.py \
	--mode 2 \
	--mode1_checkpoint ./checkpoints/mode1_final \
	--output_dir /tmp/mode2 \
	--max_event_tokens 40 \
	--d_model 1024 \
	--encoder_layers 2 \
	--decoder_layers 2 \
	--num_cap_layers 3 \
	--num_queries 30 \
	--num_train_epochs 100 \
	--per_device_train_batch_size 32 \
	--learning_rate 2e-4

2026-01-04 19:46:10.728676: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-04 19:46:10.750251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767555970.773316  353097 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767555970.780480  353097 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767555970.799991  353097 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
%%time
!python eval.py \
	--checkpoint_path ./checkpoints/mode2_final \
	--eval_val False \
	--per_device_eval_batch_size 64 \
	--max_event_tokens 40 \
	--d_model 1024 \
	--encoder_layers 2 \
	--decoder_layers 2 \
	--num_cap_layers 3 \
	--num_queries 30

2026-01-05 02:24:04.156305: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-05 02:24:04.701080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767579844.938428    7907 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767579845.001538    7907 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767579845.491756    7907 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 